<a href="https://colab.research.google.com/github/juliancramos/cnn-pneumonia-covid-detector/blob/main/cnn-pneumonia-covid-detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chest X-Ray Classification: Normal vs. Pneumonia vs. COVID-19

**Authors:** Sergio Ortíz - Julián Ramos - Melissa Ruíz

**Course:** Machine Learning Techniques

**Date:** February 2026

## 1. Executive Summary
Lung diseases such as Pneumonia and COVID-19 are critical health conditions requiring timely diagnosis. Traditional manual interpretation of chest X-rays is subjective and prone to error.

This project aims to develop an automated Deep Learning system using **Convolutional Neural Networks (CNNs)** to classify chest X-ray images into three categories:
1.  **Normal**
2.  **Pneumonia**
3.  **COVID-19**

---
### 1.1 Project Objectives
This project follows a structured workflow designed to meet specific technical requirements:

1.  **Exploratory Data Analysis (EDA):**
    * Explore the dataset to identify key characteristics, including class distribution and image quality.
    * Detect potential biases (e.g., class imbalance) and present visualizations to support design decisions.

2.  **Image Preprocessing:**
    * Implement techniques to improve input quality, such as pixel normalization and resizing.
    * Apply **Data Augmentation** strategies to improve model generalization and prevent overfitting.
    * Justify how these steps prepare the data for the neural network.

3.  **Neural Network Design:**
    * Design a custom **Convolutional Neural Network (CNN)** architecture suitable for classification.
    * Justify the selection of layers (Convolutional, Pooling, Fully Connected), filter sizes, and activation functions.
    * Present the architecture diagram and explain its expected behavior.

4.  **Training and Evaluation:**
    * Train the model using an appropriate optimization strategy and loss function.
    * Evaluate performance on a validation set using robust metrics such as **Precision, Recall, and F1-Score**.
    * Analyze results to identify issues like overfitting and propose technical solutions.

## 2. Environment Setup and Configuration

In this section, the execution environment is configured.

### 2.1 Technical Stack
* **Data Manipulation:** `pandas` and `numpy` are utilized for metadata handling and numerical operations.
* **Visualization:** `matplotlib` and `seaborn` are employed to analyze data distributions and training metrics.
* **Image Processing:** `cv2` (OpenCV) is used for reading and resizing X-ray images.
* **Deep Learning:** `tensorflow` and `keras` serve as the core framework for constructing the CNN.



In [11]:
import os
import random
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import tensorflow as tf

# CONSTANTS CONFIGURATION
# Fixed seed to ensure reproducibility (weights)
SEED = 42
# Target image resolution (224x224 pixels)
IMG_SIZE = 224
# Number of images processed per iteration
BATCH_SIZE = 32

def set_seed(seed_value):
    """
    Sets the random seed for reproducibility across all libraries.

    This function locks the pseudo-random number generators of Python, NumPy,
    and TensorFlow to a specific sequence.

    Args:
        seed_value (int): The seed number to initialize generators.
    """
    # Python's built-in hashing (dictionaries/sets)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

    # Python's standard random library (shuffle, choice)
    random.seed(seed_value)

    # NumPy's random module (array splitting, matrices)
    np.random.seed(seed_value)

    # TensorFlow's random module (weight initialization)
    tf.random.set_seed(seed_value)

# Apply the configuration
set_seed(SEED)

# Visualization Setup
# Set a style for all plots (background grid)
sns.set_style("whitegrid")

# Define a global figure size (12x6 inches)
plt.rcParams['figure.figsize'] = (12, 6)

# Silence non-critical alerts (DeprecationWarnings)
warnings.filterwarnings('ignore')

print("Environment setup complete.")
print(f"TensorFlow Version: {tf.__version__}")

Environment setup complete.
TensorFlow Version: 2.19.0


## 3. Data Ingestion

In this section, the required datasets are downloaded from Kaggle.

The Kaggle API (`kaggle.json`) is utilized because the dataset consists of **unstructured image data** organized in directory trees, rather than structured tabular files (e.g., CSV or Excel).
**Datasets acquired:**
* **Chest X-Ray Images (Pneumonia):** provided by Paul Mooney.
* **COVID-19 Radiography Database:** provided by Tawsifur Rahman.

In [14]:
import os
from google.colab import drive

def setup_kaggle_from_drive():
    """
    Configures the Kaggle API by loading the token from Google Drive.

    This function mounts the user's Google Drive, looks for 'kaggle.json'
    in the root directory ('/content/drive/MyDrive/'), and download the datasets.
    """

    # Validate if data already exists
    if os.path.exists('chest_xray') and os.path.exists('COVID-19_Radiography_Dataset'):
        print("Data already exists. Skipping download and extraction.")
        return

    # Mount Google Drive
    print("Mounting Google Drive...")
    drive.mount('/content/drive')

    # Path to the token in Drive
    drive_path = '/content/drive/MyDrive/kaggle.json'
    #local_path = '/root/.kaggle/kaggle.json'

    # Check and Copy Credentials
    if not os.path.exists(drive_path):
        print(f"Error: 'kaggle.json' not found at {drive_path}.")
        print("Upload the file to your Google Drive root folder.")
        return

    # Create the hidden directory .kaggle
    !mkdir -p ~/.kaggle

    # Copy the file from Drive to the local Colab environment
    !cp "$drive_path" ~/.kaggle/

    # Set security permissions (Required by Kaggle API)
    !chmod 600 ~/.kaggle/kaggle.json
    print("API Token loaded from Drive successfully.")

    # 3. Dataset Download & Extraction
    print("\nDownloading Datasets...")

    print("Downloading Chest X-Ray Images (Pneumonia)...")
    !kaggle datasets download -d paultimothymooney/chest-xray-pneumonia --force

    print("Downloading COVID-19 Radiography Database...")
    !kaggle datasets download -d tawsifurrahman/covid19-radiography-database --force

    print("\nExtracting Files...")
    # Unzip
    !unzip -qo chest-xray-pneumonia.zip
    !unzip -qo covid19-radiography-database.zip

    print("SUCCESS: Data ingestion complete.")

# Execute the function
setup_kaggle_from_drive()

Data already exists. Skipping download and extraction.
